In [142]:
pip install geopy

In [218]:
import pandas as pd
import numpy as np
import requests
import json
import geopy.distance as GPD
import pickle

In [110]:
lookup = pd.read_csv('data/sitetype_lookup.csv', index_col='value')
lookup.head()

,site
value,
98,Other
99,No Gate No See
A1,Accessory Building
A2,Abandoned
A3,Group Mailbox


In [109]:
lookup.columns

Index([' site'], dtype='object')

In [111]:
lookup.loc['98','site']

'Other'

In [78]:
resp = requests.get("https://gisdata.kingcounty.gov/arcgis/rest/services/OpenDataPortal/property__parcel_address_area/MapServer/1722/query?where=SITETYPE%20%3D%20'R1'&outFields=SITETYPE,LAT,LON,ZIP5,PLUS4&outSR=4326&f=json")

In [79]:
x = json.loads(resp.text)
x

{'displayFieldName': 'FULLNAME',
 'fieldAliases': {'SITETYPE': 'ADDR_TYPE',
  'LAT': 'LAT',
  'LON': 'LON',
  'ZIP5': 'ZIP5',
  'PLUS4': 'PLUS4'},
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'SITETYPE',
   'type': 'esriFieldTypeString',
   'alias': 'ADDR_TYPE',
   'length': 2},
  {'name': 'LAT', 'type': 'esriFieldTypeDouble', 'alias': 'LAT'},
  {'name': 'LON', 'type': 'esriFieldTypeDouble', 'alias': 'LON'},
  {'name': 'ZIP5',
   'type': 'esriFieldTypeString',
   'alias': 'ZIP5',
   'length': 5},
  {'name': 'PLUS4',
   'type': 'esriFieldTypeString',
   'alias': 'PLUS4',
   'length': 4}],
 'features': [{'attributes': {'SITETYPE': 'R1',
    'LAT': 47.65611332,
    'LON': -122.35805293,
    'ZIP5': '98107',
    'PLUS4': '4928'},
   'geometry': {'rings': [[[-122.357975286805, 47.65626286177175],
      [-122.35797738811061, 47.65596059402691],
      [-122.35811264309456, 47.65596033248332],
      [-122.35813363209269, 

In [86]:
'exceededTransferLimit' not in x

False

In [70]:
x['features'][0]['attributes']

{'SITETYPE': 'P4',
 'LAT': 47.62531071,
 'LON': -122.32302985,
 'ZIP5': '98102',
 'PLUS4': '4603'}

In [73]:
resp.text

'{"displayFieldName":"FULLNAME","fieldAliases":{"SITETYPE":"ADDR_TYPE","LAT":"LAT","LON":"LON","ZIP5":"ZIP5","PLUS4":"PLUS4"},"geometryType":"esriGeometryPolygon","spatialReference":{"wkid":4326,"latestWkid":4326},"fields":[{"name":"SITETYPE","type":"esriFieldTypeString","alias":"ADDR_TYPE","length":2},{"name":"LAT","type":"esriFieldTypeDouble","alias":"LAT"},{"name":"LON","type":"esriFieldTypeDouble","alias":"LON"},{"name":"ZIP5","type":"esriFieldTypeString","alias":"ZIP5","length":5},{"name":"PLUS4","type":"esriFieldTypeString","alias":"PLUS4","length":4}],"features":[]}'

In [83]:
for y in x['features']:
    print(y['attributes']['ZIP5'])

98107
98107
98107
98107
98107
98107
98107
98103
98103
98103
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98125
98198
98198
98198
98198
98198
98198
98198
98198
98198
98198
98198
98198
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98166
98055
98055
98055
98055
98055
98029
98029
98029
98029
98029
98029
98029
98029
98029
98029
98033
98033
98033
98033
98033
98033
98033
98055
98055
98055
98055
98055
98055
98055
98055
98055
98198
98006
98006
98006
98006
98006
98006
98040
9804

In [168]:
data_lst = []
for value in lookup.index:
    resp = requests.get("https://gisdata.kingcounty.gov/arcgis/rest/services/OpenDataPortal/property__parcel_address_area/MapServer/1722/query?where=SITETYPE%20%3D%20'"+value+"'&outFields=SITETYPE,LAT,LON,ZIP5,PLUS4&outSR=4326&f=json")
    x = json.loads(resp.text)
    if 'exceededTransferLimit' not in x:
        for y in x['features']:
            entry = {}
            entry['sitetype'] = lookup.loc[value,'site'].replace(' ','_')
            entry['lat'] = y['attributes']['LAT']
            entry['lon'] = y['attributes']['LON']
            entry['zip'] = y['attributes']['ZIP5']
            entry['plus4'] = y['attributes']['PLUS4']
            data_lst.append(entry)
        

In [118]:
data = pd.DataFrame(data_lst)
data.groupby('sitetype').count()

,lat,lon,zip,plus4
sitetype,,,,
Abandoned,18,18,18,15
Access_Point,9,9,9,6
Accessory_Building,395,395,395,311
Airport,10,10,10,9
Camp/Bungalow,16,16,16,10
Campground,10,10,10,7
Cemetery,54,54,54,42
Commercial_Farm,75,75,75,67
Cultural,125,125,125,118


In [214]:
to_drop = ['Group_Mailbox','No_Gate_No_See', 'Other', 'Other_Residential', 
           'Town_Boundary_Point', 'Accessory_Building', 'Camp/Bungalow', 
           'PSAP','Hanger']
keep = list(set(data.sitetype) - set(to_drop))

In [215]:
data = data[data.sitetype.isin(keep)]

In [220]:
data.to_csv('gis.csv', index=False)

In [145]:
clean = pd.read_csv('data/clean.csv')
clean.head()

,date,price,bedrooms,sqft_living,sqft_lot,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bathroomsx4,floorsx2
0,164,221900.0,3,1180,5650,0,0,3,7,1180,0,1955,1955,98178,47.5112,-122.257,1340,5650,4,2
1,221,538000.0,3,2570,7242,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9,4
2,299,180000.0,2,770,10000,0,0,3,6,770,0,1933,1933,98028,47.7379,-122.233,2720,8062,4,2
3,221,604000.0,4,1960,5000,0,0,5,7,1050,910,1965,1965,98136,47.5208,-122.393,1360,5000,12,2
4,292,510000.0,3,1680,8080,0,0,3,8,1680,0,1987,1987,98074,47.6168,-122.045,1800,7503,8,2


In [174]:
st='Police'
data.loc['sitetype'] == st


KeyError: 'sitetype'

In [217]:
row_dists = []
for e, row in enumerate(clean.iterrows()):
    print(e)
    lat1 = row[1]['lat']
    lon1 = row[1]['long']
    coords1 = (lat1, lon1)
    dist_cols = {}
    for st in keep:
        df = data[data['sitetype'] == st]
        dists = []
        for site in df.iterrows():
            lat2 = site[1].lat
            lon2 = site[1].lon
            coords2 = (lat2, lon2)
            dist = GPD.geodesic(coords1, coords2).km
            dists.append(dist)
        dist_cols[st] = min(dists)
    row_dists.append(dist_cols)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


...

15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
1567

16877
16878
16879
16880
16881
16882
16883
16884
16885
16886
16887
16888
16889
16890
16891
16892
16893
16894
16895
16896
16897
16898
16899
16900
16901
16902
16903
16904
16905
16906
16907
16908
16909
16910
16911
16912
16913
16914
16915
16916
16917
16918
16919
16920
16921
16922
16923
16924
16925
16926
16927
16928
16929
16930
16931
16932
16933
16934
16935
16936
16937
16938
16939
16940
16941
16942
16943
16944
16945
16946
16947
16948
16949
16950
16951
16952
16953
16954
16955
16956
16957
16958
16959
16960
16961
16962
16963
16964
16965
16966
16967
16968
16969
16970
16971
16972
16973
16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
1704

18243
18244
18245
18246
18247
18248
18249
18250
18251
18252
18253
18254
18255
18256
18257
18258
18259
18260
18261
18262
18263
18264
18265
18266
18267
18268
18269
18270
18271
18272
18273
18274
18275
18276
18277
18278
18279
18280
18281
18282
18283
18284
18285
18286
18287
18288
18289
18290
18291
18292
18293
18294
18295
18296
18297
18298
18299
18300
18301
18302
18303
18304
18305
18306
18307
18308
18309
18310
18311
18312
18313
18314
18315
18316
18317
18318
18319
18320
18321
18322
18323
18324
18325
18326
18327
18328
18329
18330
18331
18332
18333
18334
18335
18336
18337
18338
18339
18340
18341
18342
18343
18344
18345
18346
18347
18348
18349
18350
18351
18352
18353
18354
18355
18356
18357
18358
18359
18360
18361
18362
18363
18364
18365
18366
18367
18368
18369
18370
18371
18372
18373
18374
18375
18376
18377
18378
18379
18380
18381
18382
18383
18384
18385
18386
18387
18388
18389
18390
18391
18392
18393
18394
18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
1840

19609
19610
19611
19612
19613
19614
19615
19616
19617
19618
19619
19620
19621
19622
19623
19624
19625
19626
19627
19628
19629
19630
19631
19632
19633
19634
19635
19636
19637
19638
19639
19640
19641
19642
19643
19644
19645
19646
19647
19648
19649
19650
19651
19652
19653
19654
19655
19656
19657
19658
19659
19660
19661
19662
19663
19664
19665
19666
19667
19668
19669
19670
19671
19672
19673
19674
19675
19676
19677
19678
19679
19680
19681
19682
19683
19684
19685
19686
19687
19688
19689
19690
19691
19692
19693
19694
19695
19696
19697
19698
19699
19700
19701
19702
19703
19704
19705
19706
19707
19708
19709
19710
19711
19712
19713
19714
19715
19716
19717
19718
19719
19720
19721
19722
19723
19724
19725
19726
19727
19728
19729
19730
19731
19732
19733
19734
19735
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
1977

20975
20976
20977
20978
20979
20980
20981
20982
20983
20984
20985
20986
20987
20988
20989
20990
20991
20992
20993
20994
20995
20996
20997
20998
20999
21000
21001
21002
21003
21004
21005
21006
21007
21008
21009
21010
21011
21012
21013
21014
21015
21016
21017
21018
21019
21020
21021
21022
21023
21024
21025
21026
21027
21028
21029
21030
21031
21032
21033
21034
21035
21036
21037
21038
21039
21040
21041
21042
21043
21044
21045
21046
21047
21048
21049
21050
21051
21052
21053
21054
21055
21056
21057
21058
21059
21060
21061
21062
21063
21064
21065
21066
21067
21068
21069
21070
21071
21072
21073
21074
21075
21076
21077
21078
21079
21080
21081
21082
21083
21084
21085
21086
21087
21088
21089
21090
21091
21092
21093
21094
21095
21096
21097
21098
21099
21100
21101
21102
21103
21104
21105
21106
21107
21108
21109
21110
21111
21112
21113
21114
21115
21116
21117
21118
21119
21120
21121
21122
21123
21124
21125
21126
21127
21128
21129
21130
21131
21132
21133
21134
21135
21136
21137
21138
21139
21140
2114

In [221]:
# with open('data/distances.pickle', 'wb') as f:
#     pickle.dump(row_dists, f)

In [222]:
pd.DataFrame(row_dists)

,Police,Fire,Commercial_Farm,Campground,Seasonal_Home,Lodging,Gate_w/o_Building,Cultural,Abandoned,Public_Gathering,Government,Utility,Access_Point,Cemetery,Airport,Educational,Gated_w/_Building
0,2.456861,2.116200,3.184509,11.040112,29.369928,2.317636,31.195278,0.460860,2.136789,0.606061,1.023579,0.295342,7.275835,1.621051,3.673296,0.421127,9.560145
1,7.115889,1.622418,8.646702,20.270435,30.610873,1.660088,29.953770,0.530477,9.426786,0.602779,1.569912,1.441444,4.961174,1.883651,6.093273,0.347626,4.452579
2,2.532960,2.463073,4.486447,14.397269,25.093265,1.931967,23.298991,2.237051,11.016414,0.268421,1.219589,0.190037,9.238893,3.456310,2.836639,0.180398,5.110160
3,2.866812,1.891094,5.054405,15.601558,38.029610,3.723783,40.006992,2.741761,8.132931,0.908505,1.977436,0.147520,16.934488,3.111443,9.420195,0.405319,17.184939
4,9.485522,1.241106,4.493620,8.819946,9.752731,6.608390,11.747274,4.678520,8.815768,1.256952,0.660797,0.416444,11.145737,7.207596,14.933916,1.132304,7.132730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21229,9.626441,2.429806,5.507926,22.112902,32.082776,0.083691,32.407528,1.555726,6.807081,0.516425,0.715120,0.220297,3.404938,1.033515,7.878784,0.623718,7.533140
21230,0.863806,2.642585,2.619856,13.301577,36.349483,4.413713,38.283140,0.721808,6.131955,0.757715,0.783129,1.262395,14.821864,0.705413,10.470231,0.564096,14.632280
21231,2.663531,0.523932,1.166611,18.707302,28.789463,1.921479,30.873183,1.232718,2.302045,0.388963,0.554019,1.208858,7.728936,3.020970,4.881805,0.354845,18.138877
21232,15.612248,1.076554,0.777705,3.317056,16.815001,1.315818,18.246999,2.299058,7.189766,0.837659,0.690110,0.527445,7.607151,1.853263,12.135239,1.794700,8.255115


In [199]:
pd.Series(dist_cols)

Police                 2.456861
Fire                   2.116200
Accessory_Building     1.604300
Lodging                2.317636
Hanger                 3.291250
Cultural               0.460860
Abandoned              2.136789
Public_Gathering       0.606061
Government             1.023579
Cemetery               1.621051
Educational            0.421127
Camp/Bungalow         10.634653
Commercial_Farm        3.184509
Campground            11.040112
Seasonal_Home         29.369928
Gate_w/o_Building     31.195278
PSAP                   4.750500
Other_Residential      0.985178
Utility                0.295342
Access_Point           7.275835
Airport                3.673296
Gated_w/_Building      9.560145
dtype: float64

Note:     
**Access Points**:   Regional Access Points (RAPs) are an entry point to CEA. These entry points are resource centers where households experiencing homelessness can get help finding housing and other resources. Individuals and families experiencing homelessness may call ahead to schedule an appointment.    
**PSAP**:    A public-safety answering point (PSAP), sometimes called "public-safety access point" is a call center where emergency calls (like police, fire brigade, ambulance) initiated by any mobile or landline subscriber are terminated.